# McDonalds Diet Problem

In this notebook, we will determine the minimum cost McDonalds diet

In [ ]:
foods = [:QP, :MD, :BM, :FF, :MC, :FR, :SM, :M1, :OJ]
# Word to the wise---Julia symbols cannot start with a number (M1 instead of 1M. It took me a while to figure this out)

nutrients = [:Prot, :VitA, :VitC, :Calc, :Iron, :Cals, :Carb]

cost = Dict(zip(foods,[1.84,2.19,1.84,1.44,2.29,0.77,1.29,0.6,0.72]))
required = Dict(zip(nutrients,[55,100,100,100,100,2000,350]))
using NamedArrays
A = [
    28 24 25 14 31 3 15 9 1
    15 15 6 2 8 0 4 10 2
    6 10 2 0 15 15 0 4 120
    30 20 25 15 15 0 20 30 2
    20 20 20 10 8 2 15 0 2
    510 370 500 370 400 220 345 110 80
    34 33 42 38 42 26 27 12 20
]
A_NA = NamedArray(A, (nutrients,foods), ("Nutrients","Menu Items")) ;



Some new requirements 

Helen will allow me to eat as many burgers as a want, so I can maximize the number of burgers, but 

* We now have some maximum amount of every nutrient (say twice the minimum requirement)

* I can't let the number of calories I eat get "out of proportion" to my vitamin levels.
Specifically 

(cals eaten)/(vitc eaten) <= $\rho$

(cals eaten)/(vita eaten) <= $\rho$

for $\rho = 40$


In [ ]:
burgers = [:QP, :MD, :BM]
# Fancy dict comprehension to create max_allowed
max_allowed = Dict( x => 3*required[x] for x in keys(required))

ρ = 40 ;

In [ ]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
@variable(m, x[foods] >= 0)
@objective(m, Max, sum(x[j] for j in burgers))
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])

# Add max nutrients 
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) <= max_allowed[i])

# Let's use 'expressions'
@expression(m, cals_eaten, sum(A_NA[:Cals,j]*x[j] for j in foods))
@expression(m, vitc_eaten, sum(A_NA[:VitC,j]*x[j] for j in foods))
@expression(m, vita_eaten, sum(A_NA[:VitA,j]*x[j] for j in foods))

# Do the ratio constraints
@constraint(m, vitc_ratio, cals_eaten / vitc_eaten <= ρ)
@constraint(m, vita_ratio, cals_eaten / vita_eaten <= ρ)

print(m)

# If you don't want the solver output, you can make it slient like this
set_silent(m)
optimize!(m)

# You can use the Julia Formatting library for prettier printing, specifying precision, or you can use @printf like C
using Formatting
printfmtln("Minimum cost menu \${:.2f}: ", objective_value(m))
for j in foods
    if value(x[j]) > 1.0e-6
        printfmtln("Eat {1:.2f} of menu item {2}", value(x[j]), j)
    end
end



Oh Dear, this was not a linear program, so HiGHS could not solve it.

(cals eaten)/(vitc eaten) <= $\rho$

(cals eaten)/(vita eaten) <= $\rho$

But since vitc eaten and vita eaten are both > 0, we can multiply through by the denominator and write it as a linear program:

(cals eaten) <= (vitc eaten) * $\rho$

(cals eaten) <= (vita eaten) * $\rho$

In [ ]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
@variable(m, x[foods] >= 0)
@objective(m, Max, sum(x[j] for j in burgers))
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])

# Add max nutrients 
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) <= max_allowed[i])

# Let's use 'expressions'
@expression(m, cals_eaten, sum(A_NA[:Cals,j]*x[j] for j in foods))
@expression(m, vitc_eaten, sum(A_NA[:VitC,j]*x[j] for j in foods))
@expression(m, vita_eaten, sum(A_NA[:VitA,j]*x[j] for j in foods))

# Do the ratio constraints as LP
@constraint(m, vitc_ratio, cals_eaten <= vitc_eaten * ρ)
@constraint(m, vita_ratio, cals_eaten <= vita_eaten * ρ)


# If you don't want the solver output, you can make it slient like this
#set_silent(m)
optimize!(m)

# You can use the Julia Formatting library for prettier printing, specifying precision, or you can use @printf like C
using Formatting
printfmtln("Minimum cost menu \${:.2f}: ", objective_value(m))
for j in foods
    if value(x[j]) > 1.0e-6
        printfmtln("Eat {1:.2f} of menu item {2}", value(x[j]), j)
    end
end

